# **COMPANIES DATASETS JOIN**

> Each dataset contains the information regarding the same companies from 3 different sources. 
> The point of this project is to create a 4th dataset, containing the other 3, and by joining them, better accuracy on the common columns is desired. 
> The columns that interest us the most are Category, Address (country, region, city, address), Phone, Company names. These columns will be mapped for the new dataset created.

# Contents

* [Visually explore the data](#1)
* [Missing data](#2)
* [Keep only the most relevant columns and rename them across all the datasets / create filtered datasets](#3)
* [Duplicated rows in each dataset](#4)
* [Unique values for country](#5)
* [Clean the country column in Company dataset since some values are numerical](#6)
* [Unique values for country code to check the format (2 or 3 letters)](#7)
* [Reformat phone numbers in Company dataset](#8)
* [Dictionary with phone prefix as key and country code and country name as values](#9)
* [Missing name column in Google dataset](#10)
* [Delete rows where all other columns are empty except for name and category across all datasets](#11)
* [Add '+' at the beginning of the phone number and set it as NaN if it's not valid](#12)
* [Fill in the country/country code based on phone number if provided](#13)
* [Check if there are still rows where phone, name, category and country are not bot provided and remove them](#14)
* [Merge the datasets](#15)



<a id="1" ></a>
## **Visually explore the data**

In [1]:
import numpy as np 
import pandas as pd 

bad_lines = []

def handle_bad_lines(line):
    bad_lines.append(line)
    
file_path_fb = "../input/companies-datasets/datasets/facebook_dataset.csv"

facebook = pd.read_csv(file_path_fb, on_bad_lines = handle_bad_lines, engine = 'python')

print(f"Total bad lines encountered: {len(bad_lines)}")

def lines_number(path):
    with open(path, 'r') as file:
        total_lines = sum(1 for line in file)
        
    return total_lines

print(f"Total lines in the facebook file: {lines_number(file_path_fb)}", '\n')

# check the lines that are inconsitent
# for line in range(len(bad_lines)):
#     print(bad_lines[line], '\n')

bad_lines = []

file_path_gg = "../input/companies-datasets/datasets/google_dataset.csv"
google = pd.read_csv(file_path_gg, on_bad_lines = handle_bad_lines, engine = 'python')

print(f"Total bad lines encountered: {len(bad_lines)}")

print(f"Total lines in the google file: {lines_number(file_path_gg)}", '\n')


# for line in range(len(bad_lines)):
#     print(bad_lines[line], '\n')

bad_lines = []
file_path_wb = "../input/companies-datasets/datasets/website_dataset.csv"
company = pd.read_csv(file_path_wb, on_bad_lines = handle_bad_lines, engine = 'python')

print(f"Total bad lines encountered: {len(bad_lines)}")

print(f"Total lines in the website file: {lines_number(file_path_wb)}") 

Total bad lines encountered: 8
Total lines in the facebook file: 72081 

Total bad lines encountered: 4
Total lines in the google file: 356521 

Total bad lines encountered: 8472
Total lines in the website file: 72019


* Parsing errors encountered, which means that there are lines with inconsitencies - fields are not correctly separated by commas.
* Since the number of bad lines is almost irrelevant compared to the total number of lines (companies listed), they were removed.
* I checked the data from the website_dataset.csv file and it looks as an excel file. Because it is saved as a .csv file, I opened it into a text editor to check the delimiter / or if the file contains plain text only, noticing that the delimiter is ";".

In [2]:
facebook = pd.read_csv(file_path_fb, on_bad_lines = 'skip')
google = pd.read_csv(file_path_gg, on_bad_lines = 'skip', low_memory = False)
company = pd.read_csv(file_path_wb, sep = ';')

print("Data from Facebook: ", '\n')
print(facebook.head(3), '\n')
print("Data from Google: ", '\n')
print(google.head(3), '\n')
print("Data from a Company website: ", '\n')
print(company.head(3), '\n'*5)

Data from Facebook:  

                    domain                                            address  \
0       euro-hygiene-34.fr  134 rue entrepreneurs, za du vigné, 30420, cal...   
1  lakesidehomeservices.ca                                                NaN   
2        rossiterboats.com                                                NaN   

                                          categories       city country_code  \
0                                                NaN  calvisson           fr   
1  Appliance Repair & Maintenance|Home Builders &...        NaN           ca   
2        Boats & Yachts Dealers|Boat Tours & Cruises        NaN          NaN   

  country_name                                        description  \
0       france                                                NaN   
1          NaN                                                NaN   
2          NaN  At Rossiter Boats we build semi-custom power b...   

                    email                             

<a id="2" ></a>
## **Missing data**

In [3]:
def missing_data(df, df_name):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum() / df.shape[0] * 100).sort_values(ascending = False)
    
    result = pd.concat([total, percent], axis = 1, keys = ['Total values missing', 'Percent of missing data'])
    
    print(f"Missing data in {df_name} dataframe: \n {result} \n")
    
missing_data(facebook, 'Facebook')
missing_data(google, 'Google')
missing_data(company, 'Company')

Missing data in Facebook dataframe: 
                     Total values missing  Percent of missing data
email                              51414                72.244158
description                        43978                61.795495
zip_code                           35544                49.944497
phone_country_code                 33616                47.235376
phone                              26818                37.683196
region_code                        26605                37.383900
region_name                        26605                37.383900
city                               26600                37.376874
country_name                       26302                36.958141
categories                         16574                23.288884
address                            14477                20.342293
country_code                       13976                19.638316
page_type                              4                 0.005621
domain                                

<a id="3" ></a>
## **Keep only the most relevant columns and rename them across all the datasets / create filtered datasets**

In [4]:
# columns that I am most interested overall
columns_to_keep = [ 'name', 'category', 'country', 'country_code','region', 'city', 'zip_code', 'phone', 'address']

# rename the columns accross datasets to match
facebook.rename(columns = {'categories': 'category', 'country_name': 'country', 'region_name': 'region'}, inplace = True)
google.rename(columns  ={'country_name': 'country', 'region_name': 'region'}, inplace = True)
company.rename(columns = {'s_category': 'category', 'main_country': 'country', 'legal_name': 'name', 'main_city': 'city', 'main_region': 'region'}, inplace = True)

# create new df with specific columns
facebook_filtered = facebook[columns_to_keep]
google_filtered = google[columns_to_keep]
company_filtered = company[['name', 'category', 'region', 'city', 'country', 'phone']]

# check each dataset has the same common columns names
print(facebook_filtered.columns, '\n')
print(google_filtered.columns, '\n')
print(company_filtered.columns, '\n'*5)

# check missing data again, but for the filtered datasets

print("Missing data in the facebook dataframe: ")
print(missing_data(facebook_filtered, ''), '\n')

print("Missing data in the google dataframe: ")
print(missing_data(google_filtered, ''), '\n')

print("Missing data in the company dataframe: ")
print(missing_data(company_filtered, ''))

Index(['name', 'category', 'country', 'country_code', 'region', 'city',
       'zip_code', 'phone', 'address'],
      dtype='object') 

Index(['name', 'category', 'country', 'country_code', 'region', 'city',
       'zip_code', 'phone', 'address'],
      dtype='object') 

Index(['name', 'category', 'region', 'city', 'country', 'phone'], dtype='object') 





Missing data in the facebook dataframe: 
Missing data in  dataframe: 
               Total values missing  Percent of missing data
zip_code                     35544                49.944497
phone                        26818                37.683196
region                       26605                37.383900
city                         26600                37.376874
country                      26302                36.958141
category                     16574                23.288884
address                      14477                20.342293
country_code                 13976                19.638316
name                         

In [5]:
def unique_names(df, df_name):
    count = df['name'].nunique()
    print(f"Unique names count in {df_name} dataset: {count}", '\n')

print("DATASETS DIMENSIONS: ")
print("FACEBOOK: :", facebook.shape[0], '\n')
print("GOOGLE: ", google.shape[0], '\n')
print("COMPANY: ", company.shape[0], '\n'*2)


unique_names(facebook_filtered, 'Facebook')
unique_names(google_filtered, 'Google')
unique_names(company_filtered, 'Company')

DATASETS DIMENSIONS: 
FACEBOOK: : 71167 

GOOGLE:  346925 

COMPANY:  72018 


Unique names count in Facebook dataset: 70999 

Unique names count in Google dataset: 344398 

Unique names count in Company dataset: 30619 



In [6]:
print(facebook_filtered['phone'].sample(10))

62494             NaN
12511             NaN
54343    1.403279e+10
40557    1.613239e+10
68189    1.647572e+10
6452     1.403286e+10
44370    1.403923e+10
17708             NaN
1258     1.306783e+10
68370             NaN
Name: phone, dtype: float64


In [7]:
#facebook_filtered['phone'] = facebook_filtered['phone'].astype(str).replace('nan', np.nan)
facebook_filtered.loc[:, 'phone'] = facebook_filtered['phone'].astype(str).replace('nan', np.nan)

/tmp/ipykernel_18/1397832418.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan '16138318840.0' nan ... '12703513222.0' '15197472256.0'
 '14167924420.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  facebook_filtered.loc[:, 'phone'] = facebook_filtered['phone'].astype(str).replace('nan', np.nan)


In [8]:
print(facebook_filtered['phone'].sample(10))

33225    14507722495.0
59464    14376888022.0
61330    12899691790.0
59571    14039732582.0
46688    61894004000.0
3215     17057396666.0
58750    16479251300.0
55445              NaN
11540    14166339642.0
10301    13027602080.0
Name: phone, dtype: object


In [9]:
# convert phone numbers to string and remove the trailing .0
facebook_filtered.loc[:, 'phone'] = facebook_filtered['phone'].astype(str).str.replace(r'\.0$', '', regex = True)

# convert nan strings back to NaN values (to maintain missing data integrity)
facebook_filtered.loc[:, 'phone'] = facebook_filtered['phone'].replace('nan', np.nan)

print(facebook_filtered['phone'].sample(10))

57494            NaN
45608            NaN
42178            NaN
22762            NaN
13957            NaN
61756    15145645823
65173            NaN
54798            NaN
70802            NaN
48478    16477193550
Name: phone, dtype: object


* All the datasets together contain 490 110 rows (with data regarding companies), out of which 450 049 (keeping in mind the missing data) are not null, out of which 446 016 are unique values. This means that the final dataset will have at most ~446k companies data. The closer to this number, the better.

<a id="4" ></a>
## **Duplicated rows in each dataset**

In [10]:
def remove_duplicates(df):
    initial_rows = df.shape[0]
    df = df.drop_duplicates()
    rows_after_removal = df.shape[0]
    return initial_rows - rows_after_removal

print('Duplicated rows removed from each dataset:')
print("Duplicates removed from Facebook:", remove_duplicates(facebook_filtered))
print("Duplicates removed from Google:", remove_duplicates(google_filtered))
print("Duplicates removed from Company:", remove_duplicates(company_filtered))

Duplicated rows removed from each dataset:
Duplicates removed from Facebook: 33
Duplicates removed from Google: 55
Duplicates removed from Company: 3208


## **Ideas to implement:**
- check unique values for countries to see if the names are gramatically correct and there are no inconsitencies with the lower/upper cases
- to keep in mind: the country code may be easily mapped to country if available
- noticed inconsistencies in the website.csv file in the phone number column (for eg. 4.13E+11) - convert those strange numbers to actual phone numbers
- consider a row valid only if these are not null: name, phone, country
- if the country is NaN but the phone number is provided, fill in the country by parsing the phone numbers and getting the prefix associated with it; then possibly populate the country column
- possible request to an API to get the region / city according to the address, then I can possibly populate those columns
- the main idea is to populate as many columns as possible 


In [11]:
print(facebook_filtered.shape)
print(google_filtered.shape)
print(company_filtered.shape)

(71167, 9)
(346925, 9)
(72018, 6)


In [12]:
print(type(company_filtered))
print(company_filtered.columns)
print(company_filtered['country'].dtypes)

<class 'pandas.core.frame.DataFrame'>
Index(['name', 'category', 'region', 'city', 'country', 'phone'], dtype='object')
object


<a id="5" ></a>
## **Unique values for country**

In [13]:
# new series with nan values dropped from country column to check for unique values and make sure they are correct 
def unique_values(df, df_name, column):
    
    non_nan_values = df[column].dropna()
    unique_values = non_nan_values.unique()
    
    print(f'Unique values for {column} in {df_name}: {unique_values} \n')
    
unique_values(facebook_filtered, 'Facebook', 'country')
unique_values(google_filtered, 'Google', 'country')
unique_values(company_filtered, 'Company', 'country')

Unique values for country in Facebook: ['france' 'canada' 'australia' 'united states' 'peru' 'norway'
 'united arab emirates' 'singapore' 'netherlands' 'turkey'
 'trinidad and tobago' 'germany' 'spain' 'united kingdom' 'russia'
 'hong kong' 'south africa' 'bangladesh' 'morocco' 'malaysia'
 'south korea' 'poland' 'italy' 'gabon' 'greece' 'taiwan' 'switzerland'
 'sweden' 'romania' 'india' 'portugal' 'philippines' 'denmark' 'chile'
 'mexico' 'oman' 'new zealand' 'belgium' 'brazil' 'georgia' 'china'
 'mozambique' 'indonesia' 'ireland' 'qatar' 'pakistan' 'saudi arabia'
 'austria' 'vietnam' 'sri lanka' 'colombia' 'slovakia'
 'hashemite kingdom of jordan' 'ukraine' 'japan' 'ecuador' 'panama'
 'luxembourg' 'tajikistan' 'finland' 'liechtenstein' 'myanmar' 'kuwait'
 'egypt' 'hungary' 'israel' 'kosovo' 'thailand' 'costa rica' 'el salvador'
 'brunei' 'argentina' 'venezuela' 'bosnia and herzegovina' 'croatia'
 'estonia' 'czechia' 'mauritius' 'cambodia' 'zimbabwe' 'botswana'
 'paraguay' 'belarus' 'i

<a id="6" ></a>
## **Clean the country column in Company dataset since some values are numerical**

In [14]:
# delete rows where there are non-string values for country column in company dataset
# regex matches letters (both lowercase and uppercase), $ asserts the end of the string, ensuring the entire string follows the pattern
company_filtered = company_filtered[company_filtered['country'].str.match(r'^[a-zA-Z]+(?:[ -][a-zA-Z]+)*$') | company_filtered['country'].isna()]

# some countries were upper cased
company_filtered['country'] = company_filtered['country'].str.lower()

# noticed a 001 value in the google dataset, in country code column and removed that row
google_filtered = google_filtered[google_filtered['country_code'] != '001']

unique_v = company_filtered['country'].dropna().unique()
print("Unique values for countries in company dataset after filtering: ", unique_v)

Unique values for countries in company dataset after filtering:  ['canada' 'australia' 'germany' 'united states' 'ireland' 'malaysia'
 'united kingdom' 'italy' 'netherlands' 'taiwan' 'united arab emirates'
 'hong kong' 'brazil' 'bangladesh' 'russia' 'argentina' 'finland'
 'austria' 'india' 'japan' 'new zealand' 'chile' 'greece' 'latvia'
 'philippines' 'belgium' 'samoa' 'china' 'turkey' 'croatia' 'south africa'
 'spain' 'kenya' 'saudi arabia' 'vietnam' 'singapore' 'israel' 'qatar'
 'norway' 'france' 'thailand' 'kuwait' 'poland' 'togo' 'portugal'
 'bulgaria' 'colombia' 'costa rica' 'switzerland' 'puerto rico' 'nigeria'
 'sweden' 'czechia' 'south korea' 'afghanistan' 'iraq'
 'bosnia and herzegovina' 'peru' 'hungary' 'vn' 'ca' 'com' 'slovenia'
 'senegal' 'sri lanka' 'indonesia' 'denmark' 'papua new guinea' 'albania'
 'guinea' 'romania' 'mexico' 'tunisia' 'venezuela' 'kazakhstan'
 'mauritius' 'oman' 'cyprus' 'namibia' 'nicaragua' 'ukraine' 'algeria'
 'slovakia' 'nepal' 'montenegro' 'uruguay

<a id="7" ></a>
## **Unique values for country code to check the format (2 or 3 letters)**

In [15]:
unique_values(facebook_filtered, 'Facebook', 'country_code')
unique_values(google_filtered, 'Google', 'country_code')

Unique values for country_code in Facebook: ['fr' 'ca' 'au' 'gb' 'us' 'pe' 'de' 'no' 'ae' 'sg' 'nl' 'tr' 'tt' 'nz'
 'es' 'ru' 'hk' 'za' 'bd' 'ma' 'my' 'jp' 'kr' 'pl' 'be' 'it' 'ga' 'gr'
 'tw' 'ch' 'se' 'cr' 'ro' 'gt' 'in' 'pt' 'ph' 'dk' 'cl' 'mx' 'om' 'bs'
 'br' 'ge' 'ec' 'cn' 'qa' 'mz' 'id' 'ie' 'ar' 'pk' 'sa' 'lk' 'at' 'vn'
 'co' 'pa' 'sk' 'jo' 'mk' 'ua' 'cz' 'th' 'lu' 'tj' 'fi' 'il' 'li' 'mm'
 'kw' 'eg' 'hu' 'mo' 'xk' 'sv' 'bn' 've' 'ba' 'vg' 'hr' 'ee' 'mu' 'kh'
 'pr' 'jm' 'zw' 'iq' 'gy' 'bw' 'py' 'by' 'is' 'bg' 'lt' 'bz' 'vi' 'ng'
 'lv' 'uy' 'hn' 'bb' 'al' 'pf' 'ao' 'gi' 'rs' 'tg' 'do' 'ni' 're' 'lb'
 'uz' 'nc' 'fo' 'gu' 'si' 'md' 'mc' 'ug' 'zm' 'bh' 'np' 'mt' 'bo' 'sn'
 'sx' 'kg' 'am' 'ke'] 

Unique values for country_code in Google: ['au' 'ca' 'us' 'my' 'be' 'fr' 'gb' 'tr' 'id' 'nz' 'in' 'qa' 'de' 'jp'
 'hk' 'it' 'za' 'pl' 'ro' 'mx' 'ec' 'ug' 'pr' 'mm' 'ba' 'pk' 'mt' 'br'
 'ad' 'lu' 'bb' 'th' 'kr' 'sa' 'sg' 'nl' 'es' 'si' 'jm' 'bd' 'bj' 'lr'
 'mq' 'ru' 'tw' 'ke' 'vn' 'ae' 'ie' 'c

<a id="8" ></a>
## **Reformat phone numbers in Company dataset**

In [16]:
def check_phone_format(df, df_name, column):
    # regex pattern to match phone format: 4.31E+11
    pattern = r'^\d+\.\d{2}E\+\d{2}$'

    invalid_ph = df[df[column].astype(str).str.match(pattern, na = False)]

    if len(invalid_ph) > 0:
        
        print(f"Invalid phone numbers found in: {df_name}: {len(invalid_ph)}")
    else:
        print(f"No invalid phone numbers found in: {df_name}")

check_phone_format(facebook_filtered, 'Facebook', 'phone')
check_phone_format(google_filtered, 'Google', 'phone')
check_phone_format(company_filtered, 'Company', 'phone')

No invalid phone numbers found in: Facebook
No invalid phone numbers found in: Google
Invalid phone numbers found in: Company: 5119


In [17]:
float_n = float('4.31E+11')
print(float_n, '\n')

int_n = int(float_n)
print(int_n, '\n')

str_n = str(int_n)
print(str_n, '\n')
print(type(str_n))

431000000000.0 

431000000000 

431000000000 

<class 'str'>


* Phone numbers with a strange format (for eg. 4.31E+11) in the phone column in the company dataset will be formated thusly: convert column to float to handle scientific notation, then to int, and finally to string (without scientific notation)

In [18]:
def format_phone(df, column):
    # extract the scientific notation numbers (check if the value is a string containing E+)
    # convert them to float, then to integers
    # format the integers as strings without scientific notation
    df[column] = df[column].apply(lambda x: str(int(float(x))) if (isinstance(x, str) and 'E+' in x) else x)
    
    return df

company_filtered = format_phone(company_filtered, 'phone')

check_phone_format(company_filtered, 'Company', 'phone')

No invalid phone numbers found in: Company


In [19]:
facebook_filtered['phone'].sample(10)

65213            NaN
66997    17804548473
67217            NaN
58253            NaN
12628    14056084350
43121    61401333314
3064     17808262416
56009    14167600223
65919            NaN
4201             NaN
Name: phone, dtype: object

In [20]:
google_filtered['phone'].sample(10)

297915     +18329347034
292839     +18133504020
296880     +14319993061
300877    +441312027976
83079      +14109556666
170798     +14188788900
323877     +19282844040
186866              NaN
343294     +12894074204
327445     +14167477277
Name: phone, dtype: object

In [21]:
company_filtered['phone'].sample(10)

24939      18005324270
1727       16473409099
7009       18889656064
61429      15143791333
64886      16472959425
18359      17803959734
52769      12042829500
63206      14168440000
15901    4960000000000
5847       16103881268
Name: phone, dtype: object

In [22]:
#print(company_filtered['phone'].sample(50))

* Given the missing values and my reasoning, the phone number, country, city, category and name columns are the least absent one, which adds up to the assumption that rows are valid if these columns are filled in
* Asumming that phone numbers associated with the company names are correct, the final dataset will contain data joined given the phone number; the other columns will be filled in presumably using region/ zip code column/ country code/ city columns

<a id="9" ></a>
## **Dictionary with phone prefix as key and country code and country name as values**

In [23]:
!pip install phonenumbers
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 41.6 MB/s eta 0:00:00


In [24]:
# dictionary with phone prefix as key and country code and country name as values

import phonenumbers
import phonenumbers as pn
import pycountry

phone_dict = {
    pn.country_code_for_region(c.alpha_2): (c.alpha_2.lower(), c.name.lower())
    for c in pycountry.countries
}

# dictionary with country code as key and phone prefix as value
# ph_dict = {c.alpha_2: pn.country_code_for_region(c.alpha_2) for c in pycountry.countries}
# phone_dict = {key.lower(): value for key, value in ph_dict.items()}

print(phone_dict)

{297: ('aw', 'aruba'), 93: ('af', 'afghanistan'), 244: ('ao', 'angola'), 1: ('vi', 'virgin islands, u.s.'), 358: ('fi', 'finland'), 355: ('al', 'albania'), 376: ('ad', 'andorra'), 971: ('ae', 'united arab emirates'), 54: ('ar', 'argentina'), 374: ('am', 'armenia'), 0: ('um', 'united states minor outlying islands'), 61: ('cx', 'christmas island'), 43: ('at', 'austria'), 994: ('az', 'azerbaijan'), 257: ('bi', 'burundi'), 32: ('be', 'belgium'), 229: ('bj', 'benin'), 599: ('cw', 'curaçao'), 226: ('bf', 'burkina faso'), 880: ('bd', 'bangladesh'), 359: ('bg', 'bulgaria'), 973: ('bh', 'bahrain'), 387: ('ba', 'bosnia and herzegovina'), 590: ('mf', 'saint martin (french part)'), 375: ('by', 'belarus'), 501: ('bz', 'belize'), 591: ('bo', 'bolivia, plurinational state of'), 55: ('br', 'brazil'), 673: ('bn', 'brunei darussalam'), 975: ('bt', 'bhutan'), 267: ('bw', 'botswana'), 236: ('cf', 'central african republic'), 41: ('ch', 'switzerland'), 56: ('cl', 'chile'), 86: ('cn', 'china'), 225: ('ci', 

In [25]:
#facebook_filtered['category'].sample(10)

<a id="10" ></a>
## **Missing name column in Google dataset**

In [26]:
# print columns in google dataset where name is missing
missing_name_rows = google_filtered[google_filtered['name'].isna()]
missing_name_rows

,name,category,country,country_code,region,city,zip_code,phone,address
5954,NaN,"Executive, Legislative & Government Institutions",NaN,us,NaN,NaN,NaN,NaN,NaN
34448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75503,NaN,Middle Schools,NaN,us,NaN,NaN,NaN,NaN,NaN
83092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107259,NaN,Nursing & Personal Care,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id="11" ></a>
## **Delete rows where all other columns are empty except for name and category across all datasets**


In [27]:
# delete rows where all other columns are empty except for name and category in the datasets
conditions = (facebook_filtered['name'].notna()) & (facebook_filtered['category'].notna()) & (facebook_filtered.drop(['name', 'category'], axis = 1).isna().all(axis = 1))

facebook_filtered = facebook_filtered[~ conditions] # select the rows that do not meet the requirement 


conditions = (google_filtered['name'].notna()) & (google_filtered['category'].notna()) & (google_filtered.drop(['name', 'category'], axis = 1).isna().all(axis = 1))

google_filtered = google_filtered[~ conditions] 


conditions = (company_filtered['name'].notna()) & (company_filtered['category'].notna()) & (company_filtered.drop(['name', 'category'], axis = 1).isna().all(axis = 1))

company_filtered = company_filtered[~ conditions] 

print(facebook_filtered.shape)
print(google_filtered.shape)
print(company_filtered.shape)

(65303, 9)
(344036, 9)
(64827, 6)


* Now, all the data has name, category and at least one other column not empty. 
* But if the columns country/country code together with the phone are not provided, then it wouldn't make sense to keep those rows since we don't have information to access to get further interested in contacting the company.
* If the phone number is provided, country/country code will be filled in based on the phone_dict dictionary defined above.

<a id="12" ></a>
## **Add '+' at the beginning of the phone number and set it as NaN if it's not valid**

* In case there are other values in the phone column other than actual phone numbers.

In [28]:
# add + sign to phone numbers where there is none and check if the value is a string (isinstance(x, str)) before applying the startswith() method
# then convert the value to a string to handle nan values, as nan values are represented as floats in pandas df

def add_plus(df, column):
    
    df[column] = df[column].apply(lambda x: '+' + str(x) if (isinstance(x, str) and not x.startswith('+')) else x)
    
    
add_plus(facebook_filtered, 'phone')
add_plus(google_filtered, 'phone')
add_plus(company_filtered, 'phone')

In [29]:
company_filtered['phone'].sample(10)

23040    +64297253177
63468    +15194737500
58006    +16479985466
34633    +19052740077
67263    +19027590353
57603    +15196863075
61803    +15144881148
32113    +14167482225
20854    +14162386736
44113    +16138612245
Name: phone, dtype: object

In [30]:
facebook_filtered['phone'].sample(10)

52954    +17054222281
69492    +15874718337
22531    +12508330164
70480    +19058498111
3806     +12503747267
57876    +14032364428
34173    +16479553277
30962    +18882472777
30585    +13065312649
36334    +38548221038
Name: phone, dtype: object

In [31]:
google_filtered['phone'].sample(10)

216710     +16174577749
310727              NaN
122741     +12407403450
80027      +19029292999
15622     +601300808888
117842     +41719135300
724        +34916553008
280877     +17022193624
278032     +15053185643
216060     +15144030503
Name: phone, dtype: object

In [32]:
import re 

def valid_phone_simple(df, column):
    
    # regex to check for a leading '+' followed by digits
    pattern = r'^\+\d+'
    
    # validation check
    df[column] = df[column].apply(lambda x: x if isinstance(x, str) and re.match(pattern, x) else np.nan)
    

valid_phone_simple(facebook_filtered, 'phone')
valid_phone_simple(google_filtered, 'phone')
valid_phone_simple(company_filtered, 'phone')

In [33]:
facebook_filtered['phone'].sample(10)

56680    +19057020707
5909              NaN
58103             NaN
59915    +12268845545
20452    +17057282522
41096             NaN
58783    +15197433531
12013    +17049831101
39675             NaN
35763    +12147422504
Name: phone, dtype: object

<a id="13" ></a>
## **Fill in the country/country code based on phone number if provided**

* parse the phone number in phone column to get the country code; make sure they all start with +
* use this country code to look up the country name and country code in the phone_dict
* fill in the information in the datasets

In [34]:
# fill missing values in the country column based on the phone prefix, and overwrite them if they are not correct but match the prefix 
import phonenumbers 

def fill_missing_countries(row):
    
    if pd.isna(row['phone']):
        return row
    
    try:
        # parse the ph no
        phone_number = phonenumbers.parse(row['phone'])
        
        # extract country code from the ph no
        country_code = phone_number.country_code
        
        if country_code in phone_dict:
            # check in phone dict
            country_c, country_name = phone_dict[country_code] # extract the country code and name from the dictionry
            
            # fill missing country name or overwrite incorrect ones based on the assumptions the phone number is correct
            if pd.isna(row['country']) or row['country'] != country_name:
                row['country'] = country_name
                
            # fill missing county code or incorrect ones
            if pd.isna(row['country_code']) or row['country_code'] != country_c:
                row['country_code'] = country_c
                
    except Exception as e:
        print(f"Error processing phone number {row['phone']}: {e}")
    
    return row

print("Number of valid values in country column before mapping using phone prefix: ", facebook_filtered['country'].notna().sum(), '\n')

facebook_filtered = facebook_filtered.apply(fill_missing_countries, axis = 1)

print("Number of valid values in country column after mapping using phone prefix: ", facebook_filtered['country'].notna().sum(), '\n')

#print(facebook_filtered.sample(10))

Number of valid values in country column before mapping using phone prefix:  44865 

Number of valid values in country column after mapping using phone prefix:  57181 



* Number of valid values increased in the country column, which means that the mapping using the phone prefix worked.

In [35]:
print("Number of valid values in country column before mapping using phone prefix: ", google_filtered['country'].notna().sum(), '\n')

google_filtered = google_filtered.apply(fill_missing_countries, axis = 1)

print("Number of valid values in country column after mapping using phone prefix: ", google_filtered['country'].notna().sum())

Number of valid values in country column before mapping using phone prefix:  302233 

Number of valid values in country column after mapping using phone prefix:  336587


* I had errors with processing the country column the same way in the company dataset, therefore, I checked if there are country values missing but I did not assume that the phone prefixes were well matched with their respective countries. However, assuming that the phone number (or at least the phone prefix is correct), I checked if it matches the country value in the phone_dict (that is supposed to have been accurately generated).

In [36]:
company_filtered.shape[0]

64827

In [37]:
# check for missing values in the phone column
missing_phone = company_filtered['phone'].isna().sum()
print(f"Missing values in phone column in Company dataset: {missing_phone}")

# check for missing values in the country column
missing_country = company_filtered['country'].isna().sum()
print(f"Missing values in country column in Company dataset: {missing_country}")

Missing values in phone column in Company dataset: 3422
Missing values in country column in Company dataset: 0


In [38]:
# sample of the country and phone columns
sample_data = company_filtered[['country', 'phone']].sample(10)
print(sample_data)

             country         phone
38832         canada  +15198730875
40791  united states  +16202983271
57919         canada  +14162142253
30280  united states  +19092744555
27558         canada  +18662319633
47743      australia  +61260240568
5316          canada  +16044233266
22559         canada  +18662277973
14048  united states  +12038690302
56214         canada  +18778704880


In [39]:
# counter for mismatched phone numbers with respective countries 
mismatch_count = 0

# iterate over each row in the df
for index, row in company_filtered.iterrows():
    
    phone_number = str(row['phone'])  # convert phone number to string
    country_name = row['country']  
    
    # if the country name is in the phone_dict
    if country_name.lower() in phone_dict.values():
        # get the corresponding country code and prefix from the dict
        # _ for????
        country_code = None
        country_prefix = None
        # iterate over each key-value pair in the dict
        for code, (prefix, name) in phone_dict.items():
            if name.lower() == country_name.lower():
                country_code = code
                country_prefix = prefix  
                break
        
        # check if the phone number starts with the country prefix
        if country_prefix is not None and not phone_number.startswith(str(country_prefix)):
            mismatch_count += 1

print(f"Total mismatched phone numbers: {mismatch_count}")

Total mismatched phone numbers: 0


* It seems that the phone numbers and countries in the company dataset should look good.

In [40]:
# check if the processing error makes sense, encountered in the company dataset
phone_number = phonenumbers.parse("+13066937766")

# Get the country information
country_name = phonenumbers.region_code_for_number(phone_number)
print("Country:", country_name)

Country: CA


### Fill missing country based on region if provided - another idea, eventually not implemented because phone numbers were used instead as reference

*  reverse look up in the country_subdivisions_dict: search through the values of the dictionary for each country to find a match or partial match with the provided region
* use the corresponding key, which is the country code, to fill in the missing country

In [41]:
# subdivisions = pycountry.subdivisions.get(country_code = 'us')
# for subdivision in subdivisions:
#     print(subdivision.name)

### Dictionary to hold country code to subdivisions mapping


In [42]:
# # dictionary to hold country code to subdivisions mapping

# country_subdivisions_dict = {}

# # loop through all countries provided by pycountry
# for country in pycountry.countries:
#     # subdivisions for the current country
#     subdivisions = pycountry.subdivisions.get(country_code = country.alpha_2)
    
#     # check if there are subdivisions defined
#     if subdivisions:
#         # store the subdivisions in the dictionary, using the country code as the key
#         # convert subdivision names to lowercase for consistency
#         country_subdivisions_dict[country.alpha_2.lower()] = [sub.name.lower() for sub in subdivisions]

# # eg
# print(country_subdivisions_dict.get('us'))

In [43]:
# def fill_missing_country_based_on_region(row):
    
#     # go on only if country is missing and region is provided
#     if pd.isna(row['country']) and not pd.isna(row['region']):
        
#         region_provided = row['region'].lower()
        
#         for country_code, subdivisions in country_subdivisions_dict.items():
            
#             if region_provided in subdivisions:
#                 # if exact match found
#                 country = pycountry.countries.get(alpha_2=country_code.upper())
#                 if country:
#                     row['country'] = country.name.lower()  # the full country name via pycountry
#                 break
#             else:
#                 # check for partial match
#                 for subdivision in subdivisions:
#                     if region_provided in subdivision:
#                         # partial match found
#                         country = pycountry.countries.get(alpha_2 = country_code.upper())
#                         if country:
#                             row['country'] = country.name.lower()  # set the full country name in lowercase
#                         break
                
#     return row

# facebook_filtered = facebook_filtered.apply(fill_missing_country_based_on_region, axis = 1)
# google_filtered = google_filtered.apply(fill_missing_country_based_on_region, axis = 1)
# company_filtered = company_filtered.apply(fill_missing_country_based_on_region, axis = 1)

* Now, if phone is provided, country and country code should be filled in (but country column is prioritary).

In [44]:
print('Facebook: ')
country_count = facebook_filtered['country'].count()
print("Number of not null values in the 'country' column:", country_count)

country_code_count = facebook_filtered['country_code'].count()
print("Number of not null values in the 'country code' column:", country_code_count)

phone_count = facebook_filtered['phone'].count()
print("Number of not null values in the 'phone' column:", phone_count, '\n')

print('Google: ')
country_count = google_filtered['country'].count()
print("Number of not null values in the 'country' column:", country_count)

country_code_count = google_filtered['country_code'].count()
print("Number of not null values in the 'country code' column:", country_code_count)

phone_count = google_filtered['phone'].count()
print("Number of not null values in the 'phone' column:", phone_count, '\n')

print('Company: ')
country_count = company_filtered['country'].count()
print("Number of not null values in the 'country' column:", country_count)

phone_count = company_filtered['phone'].count()
print("Number of not null values in the 'phone' column:", phone_count)

Facebook: 
Number of not null values in the 'country' column: 57181
Number of not null values in the 'country code' column: 57192
Number of not null values in the 'phone' column: 44349 

Google: 
Number of not null values in the 'country' column: 336587
Number of not null values in the 'country code' column: 329568
Number of not null values in the 'phone' column: 314785 

Company: 
Number of not null values in the 'country' column: 64827
Number of not null values in the 'phone' column: 61405


* Finally, the number of non-null phone numbers will represent the maximum number of rows/countries/company names.

<a id="14" ></a>
## **Check if there are still rows where phone, name, category and country are not bot provided and remove them**

In [45]:
print("DATASETS DIMENSIONS BEFORE: ")
print("FACEBOOK: :", facebook_filtered.shape, '\n')
print("GOOGLE: ", google_filtered.shape, '\n')
print("COMPANY: ", company_filtered.shape, '\n'*2)

conditions_facebook = (
    ((facebook_filtered['country'].notna()) & (facebook_filtered['phone'].notna()) & (facebook_filtered['name'].notna()) & (facebook_filtered['category'].notna()))
)

facebook_filtered = facebook_filtered[conditions_facebook]


conditions_google = (
    ((google_filtered['country'].notna()) & (google_filtered['phone'].notna()) & (google_filtered['name'].notna()) & (google_filtered['category'].notna()))
)

google_filtered = google_filtered[conditions_google]


conditions_company = (
    ((company_filtered['country'].notna()) & (company_filtered['phone'].notna()) & (company_filtered['name'].notna()) & (company_filtered['category'].notna()))
)

company_filtered = company_filtered[conditions_company]

print("DATASETS DIMENSIONS AFTER: ")
print("FACEBOOK: :", facebook_filtered.shape, '\n')
print("GOOGLE: ", google_filtered.shape, '\n')
print("COMPANY: ", company_filtered.shape, '\n'*2)

DATASETS DIMENSIONS BEFORE: 
FACEBOOK: : (65303, 9) 

GOOGLE:  (344036, 9) 

COMPANY:  (64827, 6) 


DATASETS DIMENSIONS AFTER: 
FACEBOOK: : (34278, 9) 

GOOGLE:  (275279, 9) 

COMPANY:  (28828, 6) 




In [46]:
facebook_filtered['region'].unique()

array([nan, 'british columbia', 'alberta', 'ontario', 'manitoba',
       'florida', 'quebec', 'provincie utrecht', 'pennsylvania', 'ohio',
       'new mexico', 'saskatchewan', 'nouvelle-aquitaine', 'california',
       'new york', 'iowa', 'tennessee', 'new jersey', 'texas',
       'rabat-sale-kenitra', 'colorado', 'massachusetts', 'virginia',
       'seoul', 'minnesota', 'illinois', 'rangpur division', 'washington',
       'new south wales', 'indiana', 'irkutsk oblast', 'north carolina',
       'queensland', 'utah', 'georgia', 'north dakota', 'dubai',
       'new brunswick', 'michigan', 'oklahoma', 'kansas', 'missouri',
       'nova scotia', 'madeira', 'moscow', 'bavaria', 'central macedonia',
       'connecticut', 'louisiana', 'england', 'alaska', 'rio de janeiro',
       'west virginia', 'nebraska', 'istanbul', 'ile-de-france',
       'rhode island', 'auckland', 'arkansas', 'wisconsin', 'maharashtra',
       'alabama', 'flanders', 'kuala lumpur', 'north rhine-westphalia',
       'mon

In [47]:
print("Missing data in the facebook dataframe: ")
print(missing_data(facebook_filtered, ''), '\n')

print("Missing data in the google dataframe: ")
print(missing_data(google_filtered, ''), '\n')

print("Missing data in the company dataframe: ")
print(missing_data(company_filtered, ''))

Missing data in the facebook dataframe: 
Missing data in  dataframe: 
               Total values missing  Percent of missing data
zip_code                     14764                43.071358
region                        9826                28.665616
city                          9824                28.659782
address                       5070                14.790828
name                             0                 0.000000
category                         0                 0.000000
country                          0                 0.000000
country_code                     0                 0.000000
phone                            0                 0.000000 

None 

Missing data in the google dataframe: 
Missing data in  dataframe: 
               Total values missing  Percent of missing data
zip_code                     58831                21.371409
region                       29485                10.710951
city                         29385                10.674625
address    

* Missing values look as expected. There are no missings in the name, category, country and phone columns. 
* Further, a dictionary containing phone number areas asssociated with region and country could be used to fill in the region; or a request to an API would be more viable to fill in the city and region, using the zip_code and country or the address.

In [48]:
# how many phone numbers start with 1 where region is missing
filtered_df = facebook_filtered[(facebook_filtered['phone'].str.startswith('1')) & (facebook_filtered['region'].isnull())]

count = len(filtered_df)

print("Number of values where the phone number starts with '1' and the region is missing:", count)

Number of values where the phone number starts with '1' and the region is missing: 0


* US and Canada have area codes in the phone numbers that can be easily mapped, but apparently there are no regions missing on the rows where the phone numbers starts with +1, at least in facebook_filtered.

<a id="15" ></a>
## **Merge the datasets**

*  Since there are around 10 000 cities in the world, I will not pay a lot of attention on the city (since the country is more relevant in terms of the location); if there are conflicts, I will take the data from one of the datasets, if it is available; I will do the same regarding the address and the phone. 

* If there is non-nan data in the datasets, I will favour the datasets information in this order: Google, Facebook, Company. After each join, missing values will be filled in with non-null values from subsequent datasets, prioritized according to the given order (Google > Facebook > Company).

In [49]:
# merge datasets sequentially
# suffixes=('_fb', '_google') makes it clear which df each column originated from
# how='outer' performs a full outer join; it combines rows from both df, keeping all rows from both and filling in missing values with nan where there is no match
companies_joined = facebook_filtered.merge(google_filtered, on = ['name', 'category', 'country', 'region', 'city', 'address', 'phone'], how = 'outer', suffixes = ('_fb', '_gg'))
companies_joined = companies_joined.merge(company_filtered, on = ['name', 'category', 'region', 'city', 'country', 'phone'], how = 'outer', suffixes = ('' , '_company'))

# function to prioritize non-null values for each row of the df
def favour_values(row):
    
    for column in companies_joined.columns:
        
        # original column name without suffix, since the column names potentially have suffixes like _fb, _gg, and _company
        # to distinguish where the data originated from after merging
        # splitting by _ and selecting the first element [0] extracts the base part of the column name
        # removing the suffix and getting the original column name
        main_column = column.split('_')[0]
        
        # check if there's a google version of the column and it's not nan, then prioritize google data
        google_column = f"{main_column}_gg"
        
        if google_column in row.index and pd.notna(row[google_column]):
            row[main_column] = row[google_column]
            
        # else, check facebook data
        elif f"{main_column}_fb" in row.index and pd.notna(row[f"{main_column}_fb"]):
            row[main_column] = row[f"{main_column}_fb"]
            
        # lastly, check company data, which is already in the original column due to the merge suffix
        elif f"{main_column}_company" in row.index and pd.notna(row[f"{main_column}_company"]):
            row[main_column] = row[f"{main_column}_company"]
            
    return row


companies_joined = companies_joined.apply(favour_values, axis = 1)


# drop columns with suffixes after data prioritization
columns_to_drop = [col for col in companies_joined.columns if '_fb' in col or '_gg' in col or '_company' in col]
companies_joined.drop(columns = columns_to_drop, inplace = True)

# keep only the first occurrence of duplicates by name:
# - calculate the number of non-nan values for each row, except the name column itself
# - sort the df by that score in descending order
# - drop duplicates based on name and keep the first row with the highest number of valid entries

companies_joined['non_na_count'] = companies_joined.apply(lambda row: row.count() - pd.isna(row['name']), axis = 1)

companies_joined = companies_joined.sort_values(by = ['name', 'non_na_count'], ascending = [True, False])

companies_joined.drop_duplicates(subset = ['name'], keep='first', inplace = True)

# reset index
companies_joined.reset_index(drop = True, inplace = True)

# drop the non_na_count column 
companies_joined.drop(columns=['non_na_count'], inplace = True)

In [50]:
companies_joined.head

<bound method NDFrame.head of                                                      name  \
0                                          !MPACT Theatre   
1                                      # 9 Auto Sales Ltd   
2                                    #1 Entertainment LLC   
3       #1 IT Support & Services Company in Winnipeg |...   
4                                                #1 Nails   
...                                                   ...   
315658                                   🥇 Strategic Mind   
315659                          🥇Béton Surface Lanaudière   
315660  🥇MyHomeware-Specialist of Tapware, Bathroom Va...   
315661  🦋 Simplicité Santé - Soins infirmiers | prélèv...   
315662  🧵 Atelier de Couture Karine Mascouche -Sur Ren...   

                                      category               country  \
0               Theatres & Theatrical Services                jersey   
1           Automobile Dealers & Manufacturers  virgin islands, u.s.   
2                    

In [51]:
print(companies_joined.shape)

(315663, 7)


In [52]:
print("Missing data in the final dataset: ")
print(missing_data(companies_joined, ''), '\n')

Missing data in the final dataset: 
Missing data in  dataframe: 
           Total values missing  Percent of missing data
address                  41134                13.030986
region                   34708                10.995270
city                     34598                10.960423
name                         0                 0.000000
category                     0                 0.000000
country                      0                 0.000000
phone                        0                 0.000000 

None 



In [53]:
unique_names_count = companies_joined['name'].nunique()
print("Number of unique values in the 'name' column:", unique_names_count)

unique_categories_count = companies_joined['category'].nunique()
print("Number of unique values in the 'category' column:", unique_categories_count)

unique_phones_count = companies_joined['phone'].nunique()
print("Number of unique values in the 'name' column:", unique_phones_count)

Number of unique values in the 'name' column: 315663
Number of unique values in the 'category' column: 6594
Number of unique values in the 'name' column: 257182


* The number of unique values in the name column matches the number of rows in the final dataset, which means that now, there will only be unique data for each company in the companies_joined.csv file.

In [54]:
companies_joined.to_csv('companies_joined.csv', index = False)